# Simple RNN

In this notebook we consider a simple example of an RNN and used a quite artifical data generating process (if you have a better idea please contact me). 

The example has been motivated by:
http://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html. 

Other Resources for RNNs:

* http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/
* http://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* http://www.deeplearningbook.org/contents/rnn.html

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
tf.__version__

'0.12.0-rc1'

In [2]:
# Global config variables (see below)
num_steps = 40     # number of truncated backprop steps
batch_size = 200  # number of minibatches b
num_classes_in = 3   # number of classes in the input
num_classes_out = 2   # number of classes in the output
state_size = 4    # number of classes in the state
learning_rate = 0.1

# Helper functions
def one_hot(Y, max):
    d = np.zeros((len(Y),max), dtype='int32')
    for row,col in enumerate(Y):
        d[row, col] = 1
    return d    

### Definition of the task

We consider a network which predicts at each point in time a variable $\hat{y}_t$. 

### Example data  (I screama, you screama, we all screama for I screama)

We need some data to play around with RNNs. They are capable of doing quite complicated things such as language models and so on. For this example, we want to generate the data ourself. We have to come up with a process which creates $x_t$ which itself can be influcenced by events $x_{t'}$ which happend before $t$. Further, we have to come up with $y_t$ which depends on $x_t'$ for timepoints $t' \le t$. 

To keep it simple, we analyse the following quite artifical process in which the weather $x_{t'}$ for $t' \le t$ influences our stock on icecream $y_t$. We then see if the RNN is capable of reconstructing that process.

#### Definition of the simple process
The weather $x_t$ at a certain point in time $t$ has three states (sunny, rainy, cloudy), which we model as $x_t = (1,0,0)$, $x_t = (0,1,0)$, and $x_t = (0,0,1)$ repectively. We assume that the weather is completly random (of course we could model more complex scenarios). 

We have an icecream store capable of holding 2 units of icecream and we start with a full store. When it is sunny we sell one unit of icecream. We have the strange policy that we order  on unit of icecream when it's claudy. It takes 3 days to deliever the ice cream, we accept the ice cream if we do not have a full stock.

This enables us to model $y_t$ the state of the store $(1,0)$ for out of stock and $(0,1)$ for in stock. We create the one-hot-encoded data in the graph later. For now we use integers but keep in mind that the data is categorical.

In [3]:
def gen_data(size=1000000):
    Xs = np.array(np.random.choice(3, size=(size,))) #Random Weather
    Y = []
    ice = 2 #Our stock of icecream at start
    for t,x in enumerate(Xs):
        # (t-3) >= 0 the first ice cream could be delivered on day 3
        # Xs[t - 3] claudy three days before today => we ordered ice cream
        # ice < 2 not full
        if (t - 3) >= 0 and Xs[t - 3] == 1 and ice < 2: 
            ice += 1
        if x == 0: # It is sunny we therefore sell ice, if we have
            if ice > 0: # We have ice cream
                ice -= 1
        if ice > 0: #We are not out of stock
            Y.append(1)
        else:
            Y.append(0)
    return Xs, np.array(Y)

In [4]:
X_train, Y_train = gen_data(50000) #Global variables holding the input and output
(X_train[0:50], Y_train[0:50])

(array([2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 0, 2, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 0, 0, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2,
        1, 0, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1]))

### Forward pass in numpy
To better illustrate the used method, we first do a forward-pass of the RNN using numpy. We load the weights which we calculated previously. Later in this notebook I will show you how to calculate these weights using TensorFlow.

In [5]:
W_, b_, V_, bv_ = np.load('rnn_weights.npy')
[W_,b_,V_,bv_]

[array([[ 0.92680091,  0.98910332,  0.33029664, -0.01007991],
        [ 0.57347268,  0.04957261, -0.11177637, -0.02239575],
        [ 0.31854928,  3.47796154,  0.6752221 , -0.1132215 ],
        [ 0.04398985,  3.33533359,  2.66867995,  1.53635991],
        [ 1.1604836 ,  0.64345938, -0.85938835,  1.47645962],
        [-0.33037862, -0.4374488 ,  2.6568048 , -3.27227044],
        [-0.01889232, -0.23109317, -2.0481782 ,  1.5775584 ]], dtype=float32),
 array([-0.31031489,  0.50388539, -0.94298124,  0.23187537], dtype=float32),
 array([[ 3.27119613, -2.98498511],
        [-1.31582463, -0.20585871],
        [ 0.15490292, -0.97352707],
        [ 0.38828596,  0.28713977]], dtype=float32),
 array([-0.24957803,  0.24958369], dtype=float32)]

### Architekture of the network 
We now define the network, we do not consider the output nodes yet.
A single RNN cell is shown in the figure below in the middle:

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)
Image taken from: [Colah's RNN Blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

The joining of the two lines coming from the previous state $h_{t-1}$ and the current x-values $x_t$ is a concantination to a vector  $[h_{t-1}, x_{t}]$ of size `state_size + num_classes_in`. Alternatively, instead of concatinating, one could also use two matrices $W_x$ and $W_h$ and keep the states seperate. This is mathematically completely identical. The new state $h_t$ is 
then calculated as:

$$
    h_{t} = \tanh([h_{t-1}, x_{t}] \cdot W + b) = \tanh(h_{t-1} \cdot W_h + x_{t} \cdot U + b)
$$

In [6]:
W_, b_

(array([[ 0.92680091,  0.98910332,  0.33029664, -0.01007991],
        [ 0.57347268,  0.04957261, -0.11177637, -0.02239575],
        [ 0.31854928,  3.47796154,  0.6752221 , -0.1132215 ],
        [ 0.04398985,  3.33533359,  2.66867995,  1.53635991],
        [ 1.1604836 ,  0.64345938, -0.85938835,  1.47645962],
        [-0.33037862, -0.4374488 ,  2.6568048 , -3.27227044],
        [-0.01889232, -0.23109317, -2.0481782 ,  1.5775584 ]], dtype=float32),
 array([-0.31031489,  0.50388539, -0.94298124,  0.23187537], dtype=float32))

In [7]:
# The first state
h0 = np.zeros(state_size) #We start with 0 initial state
x1 = one_hot(X_train, num_classes_in)[0] #Make a vector
h1 = np.tanh(np.matmul(np.concatenate([h0, x1]), W_) + b_)
h1

array([-0.31780824,  0.26622108, -0.99496676,  0.94777428])

We could now chain many of those hidden states to get a sequence of hidden states. 

In [8]:
def rnn_forward(state, X_train):
    hs = []
    for t in range(len(X_train)):
        state = np.tanh(np.matmul(np.concatenate([state, X_train[t,:]]), W_) + b_)
        hs.append(state)
    return hs

In [9]:
rnn_forward(h0, one_hot(X_train[0:5],3))

[array([-0.31780824,  0.26622108, -0.99496676,  0.94777428]),
 array([ 0.40785938,  0.49818128, -0.07944734,  0.99714014]),
 array([ 0.33907477,  0.99889529, -0.29563198,  0.99746706]),
 array([ 0.46806674,  0.99460516, -0.48424435,  0.99753916]),
 array([ 0.94068628,  0.99729681,  0.51989672,  0.99710797])]

We add some output. For each time step the output is produced by multiplying the hidden state with:

$o_t = h_t \cdot V + b_{\tt{v}}$

This is a logit, the final the probability of output class is the softmax of the logit.

In [10]:
o1 = np.matmul(h1, V_) + bv_
prob_1 = np.exp(o1)/np.sum(np.exp(o1))
o1, prob_1

(array([-1.42560719,  2.38420341]), array([ 0.02167228,  0.97832772]))

In [11]:
h = rnn_forward(h0, one_hot(X_train,3))
pt = []
for t in range(len(h)):
    ot = np.matmul(h[t], V_ + bv_)
    pt.append(np.exp(ot)/np.sum(np.exp(ot)))

In [12]:
pt[0:10], np.argmax(pt[0:30],axis=1), Y_train[0:30]

([array([ 0.03692147,  0.96307853]),
  array([ 0.75017506,  0.24982494]),
  array([ 0.44069815,  0.55930185]),
  array([ 0.59687645,  0.40312355]),
  array([ 0.97682111,  0.02317889]),
  array([ 0.98570032,  0.01429968]),
  array([ 0.96984549,  0.03015451]),
  array([ 0.97489728,  0.02510272]),
  array([ 0.93362331,  0.06637669]),
  array([ 0.97703468,  0.02296532])],
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]))

In [13]:
np.average(np.argmax(pt, axis=1) == Y_train)

0.98702000000000001

In [14]:
tot_loss = 0
for i in range(len(Y_train)):
    tot_loss += -np.log(pt[i][Y_train[i]])

In [15]:
tot_loss / len(Y_train)

0.075494635474219834

## Calculation and Training in TensorFlow
### Preparation of the Minibatch

In this example, we have in principle a large stream of data $x$ and $y$. For efficiency reason we split the stream in minibatches of a certain length. For this task we could also imagin to have several realizations of that icecream process, so that it would also be natural to split the process into mini batches. 

For simplicyty we create the minibatch by we randomly cutting out `batch_size` entries of fixed length `num_steps`. Other, more advanced ways of doing so are possible. See e.g. https://danijar.com/variable-sequence-lengths-in-tensorflow/. For the time being, we thus consider the input tensor $X_{btc}$ for the minibatch to be of the following form:

* $b$ having `batch_size` entries
* $t$ loops over the unrolled timestamps (`num_steps`)
* $c$ has the dimension of the one-hot-coded classes (the one-hot-encoding will be done in the graph)

In [16]:
def get_batch(Xs, Ys, batch_size = 32, num_steps = 50):
    data_x = np.zeros([batch_size, num_steps], dtype=np.int32)
    data_y = np.zeros([batch_size, num_steps], dtype=np.int32)
    for i in range(1,batch_size):
        s = int(np.random.uniform(0, len(Xs) - num_steps))
        data_x[i] = Xs[s : s + num_steps]
        data_y[i] = Ys[s : s + num_steps]  
    return data_x, data_y

In [17]:
X, Y = get_batch(X_train, Y_train,3, 10)
print X
print Y

[[0 0 0 0 0 0 0 0 0 0]
 [0 1 2 2 2 2 1 1 1 1]
 [1 0 0 1 2 1 0 1 0 1]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 1]
 [1 0 0 1 1 1 1 1 1 1]]


#### Definition of the in- and outputs

We define the input and output for the graph

In [18]:
tf.reset_default_graph()
tf.set_random_seed(42)

# Placeholders
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

# RNN Inputs
# One hot encoding.
x_one_hot = tf.one_hot(x, num_classes_in)
# We want the following dimensions [batch_size, Max_Length, num_classes_in]
rnn_inputs = tf.transpose(x_one_hot, perm=(0,1,2))
rnn_inputs

<tf.Tensor 'transpose:0' shape=(200, 40, 3) dtype=float32>

The input tensor $r_{btc}$  is indexed by 

* $b$ having `batch_size` entries
* $t$ loops over the unrolled timestamps
* $c$ has the dimension of the one-hot-coded classes

#### Definition the cell

We now define the network, we do not consider the output nodes yet.
As above, in the forward pass, the new state $h_t$ is then calculated as:

$$
    h_{t} = \tanh([h_{t-1}, x_{t}] \cdot W + b) 
$$

Note that we share the variables W and b during different time-points. To do this in TensorFlow we define them in a first step for later reuse. 

In [19]:
# Definition of the Variables needed in a single cell
with tf.variable_scope('rnn_cell', reuse = False):
    W = tf.get_variable('W', [num_classes_in + state_size, state_size])
    b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))
    
# Definition of a single cell
def rnn_cell(rnn_input, state):
    with tf.variable_scope('rnn_cell', reuse=True):
        W = tf.get_variable('W', [num_classes_in + state_size, state_size])
        b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))
    return tf.tanh(tf.matmul(tf.concat(1, [state, rnn_input]), W) + b)

#### Unrolling the timesteps

We build the network using the identical weights in the `num_steps` unrolled timesteps. Techniqualy the `rnn_cell` at time t, gets the input (weather) at time t and the pervious state.  

In [20]:
state = init_state
rnn_outputs_l = []

for t in range(num_steps):
    state = rnn_cell(rnn_inputs[:,t,:], state) #Pervious state
    rnn_outputs_l.append(state) #We put the states h_0, h_1, ... in a list. 

rnn_outputs = tf.pack(rnn_outputs_l, axis=1)
rnn_outputs

<tf.Tensor 'pack:0' shape=(200, 40, 4) dtype=float32>

### Adding output to the network

We now add at each timepoint an output layer to the network. This output use the same weiht for each timepoint.

The output of the rnn is a tensor index by $o_{btk}$ (batch, time, class). This tensor produces for each minibatch and timepoint a number of output states dimensional vector indexed by $k$. This output needs to be compared with the y-value with has the shape $y_{bt}$. It's easier later on to flatten this vector for each batch.    

In [21]:
#reshape rnn_outputs and y so we can get the logits in a single matmul
rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
y_reshaped = tf.reshape(y, [-1])
y_reshaped, rnn_outputs

(<tf.Tensor 'Reshape_1:0' shape=(8000,) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(8000, 4) dtype=float32>)

In [22]:
with tf.variable_scope('softmax'):
    V = tf.get_variable('V', [state_size, num_classes_out])
    bv = tf.get_variable('b', [num_classes_out], initializer=tf.constant_initializer(0.0))

logits = tf.matmul(rnn_outputs, V) + bv
total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [23]:
# Looks quite ugly
# writer = tf.train.SummaryWriter("/tmp/dumm/RNN2", tf.get_default_graph(), 'graph.pbtxt') 

### Training

In [24]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
count = 0
sum_tr_losses = 0
for i in range(1000):
    X, Y = get_batch(X_train, Y_train, batch_size, num_steps)
    tr_losses, _ = \
    sess.run([total_loss, train_step], feed_dict={x:X, y:Y})
    sum_tr_losses += tr_losses
    count += 1
    if (i < 10) or (i % 200 == 0):
        print "{} {}".format(i, sum_tr_losses / count)
        count = 0
        sum_tr_losses = 0

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 0.762800216675
1 0.637381494045
2 0.638357937336
3 0.616243779659
4 0.617295026779
5 0.637718558311
6 0.58663725853
7 0.570157587528
8 0.518701076508
9 0.467967420816
200 0.205665464879
400 0.179076040313
600 0.174017876834
800 0.161909760237


### Getting the weights
We store the weights, for the numpy forward pass.

In [25]:
W_ = sess.run(W)
b_ = sess.run(b)
V_ = sess.run(V)
bv_= sess.run(bv)
print (W_,b_,V_,bv_)
np.save('rnn_weights', [W_,b_,V_,bv_])

(array([[ 0.92680091,  0.98910332,  0.33029664, -0.01007991],
       [ 0.57347268,  0.04957261, -0.11177637, -0.02239575],
       [ 0.31854928,  3.47796154,  0.6752221 , -0.1132215 ],
       [ 0.04398985,  3.33533359,  2.66867995,  1.53635991],
       [ 1.1604836 ,  0.64345938, -0.85938835,  1.47645962],
       [-0.33037862, -0.4374488 ,  2.6568048 , -3.27227044],
       [-0.01889232, -0.23109317, -2.0481782 ,  1.5775584 ]], dtype=float32), array([-0.31031489,  0.50388539, -0.94298124,  0.23187537], dtype=float32), array([[ 3.27119613, -2.98498511],
       [-1.31582463, -0.20585871],
       [ 0.15490292, -0.97352707],
       [ 0.38828596,  0.28713977]], dtype=float32), array([-0.24957803,  0.24958369], dtype=float32))


### Testing 

In [26]:
preds = tf.nn.softmax(logits)
X, Y = get_batch(X_train, Y_train, batch_size, num_steps)
p_y_pred = sess.run(preds, feed_dict={x:X}) #A list for each time point
loss_train = sess.run(total_loss, feed_dict={x:X, y:Y})

In [27]:
loss_train

0.14864911

In [28]:
np.sum(np.argmax(p_y_pred, axis=1) == np.reshape(Y, -1)) / (1.0 * batch_size * num_steps)
#0.95837

0.95837499999999998

In [29]:
sess.close()

### Task
Change the state size to 2. What do you obsereve? Give an explanation for your observation. 

## Using the TensorFlow API

Alternatively one can use the TensorFlow-API for creating RNNs. In principle there are two TensorFlow methods. The first, kind of deprecated one, builds a graph from the unrolled network. This API has issues in performance, first of all the creation of the graph takes quite some time. Further, and this is a bit it is also slower during runtime. Therefore, the novel dynamic API should be prefered. If you want to use sequences of variable length see:  https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [30]:
tf.reset_default_graph()
tf.set_random_seed(42)
# Placeholders
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

# RNN Inputs
# One hot encoding.
x_one_hot = tf.one_hot(x, num_classes_in)
# We want the following dimensions [batch_size, Max_Length, num_classes_in]
rnn_inputs = tf.transpose(x_one_hot, perm=(0,1,2))
rnn_inputs

<tf.Tensor 'transpose:0' shape=(200, 40, 3) dtype=float32>

In [31]:
#cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
init_state = cell.zero_state(batch_size, tf.float32)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

In [32]:
rnn_outputs

<tf.Tensor 'RNN/transpose:0' shape=(200, 40, 4) dtype=float32>

The output $o_{btk}$ tensor produces for each minibatch and timepoint a 4 (number of output states) dimensional vector indexed by $k$. This can be compared with the y-value with has the shape $y_{bt}$.  

In [33]:
#reshape rnn_outputs and y so we can get the logits in a single matmul
rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
y_reshaped = tf.reshape(y, [-1])
y_reshaped, rnn_outputs

(<tf.Tensor 'Reshape_1:0' shape=(8000,) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(8000, 4) dtype=float32>)

In [34]:
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes_out])
    b = tf.get_variable('b', [num_classes_out], initializer=tf.constant_initializer(0.0))

logits = tf.matmul(rnn_outputs, W) + b

In [35]:
total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [36]:
Y = None
X = None
count = 0
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    training_losses = []
    for i in range(1000):
        X, Y = get_batch(X_train, Y_train, batch_size, num_steps)
        tr_losses, _ = sess.run([total_loss, train_step], feed_dict={x:X, y:Y})
        count += 1
        sum_tr_losses += tr_losses
        if (i < 10) or (i % 200 == 0):
            print "{} {}".format(i, sum_tr_losses / count)
            count = 0
            sum_tr_losses = 0

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 31.8328100145
1 0.675710082054
2 0.621600627899
3 0.605560064316
4 0.52741008997
5 0.481956064701
6 0.477577775717
7 0.446434468031
8 0.401866525412
9 0.401105493307
200 0.213907159358
400 0.132882450968
600 0.114329252839
800 0.111322681755
